Test for gmail API reader
Following instructions on that page: https://developers.google.com/gmail/api/quickstart/python

In [1]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

In [3]:
SCOPES = ["https://www.googleapis.com/auth/gmail.readonly"]

In [4]:
creds = None
json_file = "token.json"
if os.path.exists(json_file):
    creds = Credentials.from_authorized_user_file(json_file, SCOPES)

# Create my credentials (token.json) if needed

In [5]:
if creds is None:
    flow = InstalledAppFlow.from_client_secrets_file("credentials.json", SCOPES)
    creds = flow.run_local_server(port=8080, bind_addr='0.0.0.0')
    with open("token.json", "w") as token:
        token.write(creds.to_json())

>Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=1085373024194-n8rtj5ob0b8a6r96am65f846r0m1m6je.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.readonly&state=urFUzAv4jK8VyBBsHX27XY0KEEq9h9&access_type=offline                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

# Test the Gmail API

In [6]:
from googleapiclient.discovery import build

# Build the Gmail API service
service = build('gmail', 'v1', credentials=creds)

In [7]:
results = service.users().messages().list(userId='me').execute()
messages = results.get('messages', [])
print(f"By default, you only receive {len(messages)} messages.")

By default, you only receive 1 messages.


In [8]:
results = service.users().messages().list(userId='me', maxResults=120).execute()
messages = results.get('messages', [])
print(f"Now we have received {len(messages)} messages.")

Now we have received 1 messages.


In [12]:
results = service.users().messages().list(userId='me', maxResults=4).execute()
messages = results.get('messages', [])
for msg in messages:
    print('='*10)
    msg_id = msg['id']
    message = service.users().messages().get(userId='me', id=msg_id).execute()

    # Extract headers
    headers = message['payload']['headers']
    subject = next(header['value'] for header in headers if header['name'] == 'Subject')
    sender = next(header['value'] for header in headers if header['name'] == 'From')
    date = next(header['value'] for header in headers if header['name'] == 'Date')

    print(f"Subject: {subject}")
    print(f"From: {sender}")
    print(f"Date: {date}")

Subject: Re: TLDR AI - Confirm your signup
From: Ben <automatic.newsletters@gmail.com>
Date: Thu, 28 Nov 2024 22:32:57 -0500
Subject: TLDR AI - Confirm your signup
From: TLDR <dan@tldrnewsletter.com>
Date: Fri, 29 Nov 2024 03:30:55 +0000
Subject: Ben, finish setting up your new Google Account
From: Google <no-reply@google.com>
Date: Thu, 28 Nov 2024 19:07:52 -0800


In [15]:
#query = 'after:2023/01/01 before:2023/12/31 from:example@example.com'
query = 'from:dan@tldrnewsletter.com'
results = service.users().messages().list(userId='me', q=query, maxResults=4).execute()
messages = results.get('messages', [])
for msg in messages:
    print('='*10)
    msg_id = msg['id']
    message = service.users().messages().get(userId='me', id=msg_id).execute()

    # Extract headers
    headers = message['payload']['headers']
    subject = next(header['value'] for header in headers if header['name'] == 'Subject')
    sender = next(header['value'] for header in headers if header['name'] == 'From')
    date = next(header['value'] for header in headers if header['name'] == 'Date')
    email_id = msg['id']
    

    print(f"Subject: {subject}")
    print(f"From: {sender}")
    print(f"Date: {date}")
    print(f"ID: {email_id}")

Subject: TLDR AI - Confirm your signup
From: TLDR <dan@tldrnewsletter.com>
Date: Fri, 29 Nov 2024 03:30:55 +0000
ID: 19375f8c0ee524ef


In [16]:
# Get a single message
message = service.users().messages().get(userId='me', id=email_id).execute()

In [17]:
message

{'id': '19375f8c0ee524ef',
 'threadId': '19375f8c0ee524ef',
 'labelIds': ['CATEGORY_UPDATES', 'INBOX'],
 'snippet': 'Welcome to TLDR AI! Before we get into what to expect, one quick thing: 1) Reply &#39;OK&#39; to this email. (You can skip this step if you&#39;re signing up an RSS reader, Google Group, or Slack Group',
 'payload': {'partId': '',
  'mimeType': 'text/html',
  'filename': '',
  'headers': [{'name': 'Delivered-To',
    'value': 'automatic.newsletters@gmail.com'},
   {'name': 'Received',
    'value': 'by 2002:a9d:76d9:0:b0:71d:4839:1192 with SMTP id p25csp245947otl;        Thu, 28 Nov 2024 19:30:56 -0800 (PST)'},
   {'name': 'X-Google-Smtp-Source',
    'value': 'AGHT+IEbgze93xowy6tjqkwAt2lqcYGNJN0rE8gDbKkOo7vK3XcxJp3VMxD3DTq3+zp2PU9smQJI'},
   {'name': 'X-Received',
    'value': 'by 2002:a05:690c:6c0c:b0:6ef:53bb:a83c with SMTP id 00721157ae682-6ef53bbaac1mr40057687b3.20.1732851056656;        Thu, 28 Nov 2024 19:30:56 -0800 (PST)'},
   {'name': 'ARC-Seal',
    'value': 'i=1

In [ ]:
# need to parse it